## Vizualizarea Heatmap-urilor activari clasei

Heatmap ale activarilor clasei sunt de mare ajutor in identificarea partilor unei imagini care au condus CNN la clasificarea finala. Devine foarte important atunci cand analizam datele clasificate gresit.

Vom folosi o implementare a map de activare a clasei (CAM) care a fost utilizata in lucrarea, [**"Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization"**](https://arxiv.org/abs/1610.02391)

Se incarca VGG16 cu weights pretinse

In [ ]:
from keras.applications import VGG16
from keras import backend as K
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input, decode_predictions

model = VGG16(weights='imagenet')

incarca imaginea de test cu tigrul

In [ ]:
imp_path = ''

img1 = image.load_img(imp_path)
plt.imshow(img1)

img = image.load_img(img_path, target_size=(224, 224))

x= image.img_to_array(img)
x= np.expand_dims(x, axis=0)
x = preprocess_input(x)


Obtine predictiei ImageNet VGG16 pentru imaginea de intrare

In [ ]:
preds = model.predict(x)
print('Predicted:', decode_predictions(preds, top=3)[0])

Obtinerea indicelui de clasa

In [ ]:
np.argmax(preds[0])

Iesiti map caracteristicilor stratului block4_conv3, ultimul strat convolutiv din VGG16

In [ ]:
tiger_output = model.output[:, 292]
last_conv_layer = model.get_layer('block5_conv3')

#Gradients ai clasei Tiger wrt la fisierul block5_conv3
grads = K.gradients(tiger_output, last_conv_layer.output)[0]

# Fiecare intrare este intesitatea medie a gradientului in functie de un anumit canal de caracterisici map
pooled_grads = K.mean(grads, axis=(0, 1, 2))

#Aceseaza valorile pe care le-am definit avand in vedere imaginea de proba
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

#Valorile pooled_grads_value, conv_layer_output_value date imaginii noastre de intrare
pooled_grads_value, conv_layer_output_value = iterate([x])

#Inmultim fiecare canal din matricea  caracteristici cu "inportanta"
#a acestui canal in ceea ce privind imaginea de intrare
for i in range(512):
    #Mijlocul înțelept al canalului din harta funcțională rezultată este Cartea de Vânzare a CAM
    conv_layer_output_value[:, :, i] *=pooled_grads_value[i]

heatmap = np.mean(conv_layer_output_value, axis=-1)


Ploting out Heat-Map

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)


In [ ]:
import cv2

img cv2.imread(img_path)
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
heatmap = np.uint8(255 * heatmap)
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
superimposed_img = heatmap * 0.4 + img

save_img_path = ''

cv2.imwrite(save_img_path, superimposed_img)

img1 = image.load_img(save_img_path)
plt.imshow(img1)